### En este script pruebo el modelo entrenado:

1. Armo un data frame con las fechas desde hoy y hasta fin de mes
2. Creo el valor de Año, Mes y Semana
3. Pruebo el modelo


In [1]:
#Importo las librerias

import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime
from math import ceil

In [2]:
#Fijo la fecha de hoy como inicio

#start = date.today()

start= date(2023,2,1)

#Traigo el día 1 del mes siguiente
next_month = start.replace(day=28) + timedelta(days=4)

#Calculo el fin de mes

end_month = next_month - timedelta(days=next_month.day)

#Armo una secuencia de fechas entre hoy y el fin de mes
fechas = pd.date_range(start,end_month)

#Convierto el archivo en data frame
fechas = fechas.to_frame(index=False, name= 'Fechas')



In [3]:
#Creo una función para calcular la semana del mes. Las funciones por default calcular la semana del año.

def week_of_month(dt):
    """ Retorna la semana del mes para una fecha especificada.
    """ 
    
    # Fijo el dia inicial del mes
    first_day = dt.replace(day=1)
    # Seteo el dia de la semana de la fecha especificada
    dom = dt.day
    
    # Sumo  el dia de la semana de la fecha especificada + el valor de dia de la semana
    adjusted_dom = dom + first_day.weekday()

    # Dvido por 7 y redondeo
    return int(ceil(adjusted_dom/7.0))

In [4]:
#Creo los parámetros del modelo: Año, Mes y Semana

fechas['Año'] = pd.DatetimeIndex(fechas['Fechas']).year

fechas['Mes'] = pd.DatetimeIndex(fechas['Fechas']).month

fechas['Semana']=fechas['Fechas'].apply(week_of_month)

fechas = fechas.drop(['Fechas'],axis= 1)



In [5]:
#Por último me quedo con valores únicos

fechas = fechas.drop_duplicates(subset=['Año','Mes','Semana'])
fechas

,Año,Mes,Semana
0,2023,2,1
5,2023,2,2
12,2023,2,3
19,2023,2,4
26,2023,2,5


In [6]:
import pickle

#Plantamos la semilla aleatoria

np.random.seed(123)

filename = 'ut_BancoEstado.pkl'

loaded_model = pickle.load(open(filename, 'rb'))

ut_predichos = loaded_model.predict(fechas)

fechas["UT"] = np.around(ut_predichos,decimals=2)


In [7]:
fechas

,Año,Mes,Semana,UT
0,2023,2,1,19029.990234
5,2023,2,2,7405.000000
12,2023,2,3,3816.260010
19,2023,2,4,2121.000000
26,2023,2,5,1265.369995


### Obtenemos métricas de error usando datos reales como validación

In [8]:
#Importo las librerias

import pyodbc
import os

user = os.environ['USER_DW']
passw = os.environ['PASS_DW']
server = '10.0.55.50'
database = 'bi-test'

conn = pyodbc.connect('Driver={SQL Server};'
                    'Server='+server+';'
                     'Database=bi-test;'
                     'Trusted_Connection=no;'
                     'Integrated_Security=false;'
                     'UID='+user+';'
                     'PWD='+passw+';')

In [9]:
test = pd.read_sql_query("select * from VW_FIN_PROYECCION_METRICAS where Project_Name = 'Banco_Estado' and \
Año = '2023' and Mes = '2' and Tipo = 'UT'", conn)

test
                         
                        

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Project_Name,Tipo,Año,Mes,Semana,Cantidad
0,Banco_Estado,UT,2023,2,1,19030
1,Banco_Estado,UT,2023,2,2,26435
2,Banco_Estado,UT,2023,2,3,30472
3,Banco_Estado,UT,2023,2,4,32593
4,Banco_Estado,UT,2023,2,5,33742


In [ ]:
#Desagrego los valores del dataframe test

#Los datos de cantidad están agregados, por eso debo restar los valores semanales

df['lag'] = df.groupby(['Año','Mes'])['Cantidad'].shift(1)
df['Q_Mensual'] = df['Cantidad']-df['lag']

df['Q_Mensual'] = np.where(
    df['lag'].isnull(), 
    df['Cantidad'], df['Q_Mensual'])

#Me quedo con ciertas variables

df = df[['Project_Name','Tipo','Año','Mes','Semana','Q_Mensual']]
df.rename(columns={'Q_Mensual':'Cantidad'}, inplace= True)

df

In [33]:
from sklearn.metrics import mean_squared_error
from math import sqrt #Para obtener la raiz cuadrada


rmse = sqrt(mean_squared_error(test['Cantidad'], ut_predichos,squared=False))

print("RMSE")
print("--------------------------------------------")
print("El RMSE obtenido es:", round(rmse,2))
print("--------------------------------------------")


RMSE
--------------------------------------------
El RMSE obtenido es: 29.68
--------------------------------------------
